# This dataset does not contain multiple purchases from the same ID. IF it did I would have created tiers based on how much customers fly and their specific ratings.

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../data/processed/flight_customer.db")

pd.read_sql("SELECT * FROM all_customers_loyalty", conn)


,id,gender,customer_type,age,type_of_travel,class,flight_distance,inflight_wifi_service,departure/arrival_time_convenient,ease_of_online_booking,...,inflight_entertainment,on_board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,...,2,2,2,2,4,2,4,0,20.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,94171,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
129483,73097,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
129484,68825,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
129485,54173,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


### Can customer_type be used directly to simulate loyalty tiers?

In [2]:
ctlt = """
SELECT 
    ID,
    customer_type
FROM 
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
LIMIT 5
"""
customer_type_loyalty_tiers = pd.read_sql(ctlt, conn)
customer_type_loyalty_tiers

,id,customer_type
0,19556,Loyal Customer
1,90035,Loyal Customer
2,77959,Loyal Customer
3,36875,Loyal Customer
4,39177,Loyal Customer


### Analysis

No not directly. We would have to pull from some more categories to build out a loyalty tier list.

---

### Can type_of_travel (Business/Personal) represent frequency?

In [3]:
bpf = """ 
SELECT 
    type_of_travel, 
    COUNT(CASE WHEN type_of_travel = 'Business travel' THEN 1.0 ELSE 0 END) AS business_travel
FROM 
    all_customers_loyalty
GROUP BY 
    type_of_travel
"""
business_vs_personal_frequency = pd.read_sql(bpf, conn)
business_vs_personal_frequency

,type_of_travel,business_travel
0,Business travel,89445
1,Personal Travel,40042


### Analysis

Business travelers account for 69% of all trips (89,445 vs 40,042), indicating roughly a 2.2x higher frequency than personal travelers.

---

### Can flight_distance stand in for “value”?

In [4]:
fdv = """ 
SELECT 
    flight_distance,
    id
FROM 
    all_customers_loyalty
"""

flight_distance_value = pd.read_sql(fdv, conn)
flight_distance_value

,flight_distance,id
0,160,19556
1,2863,90035
2,192,12360
3,3377,77959
4,1182,36875
...,...,...
129482,192,94171
129483,2347,73097
129484,1995,68825
129485,1000,54173


### Analysis

No. We have to decide how we want to define value. Should it be based on value of ticket prices for the business or value for the customer and what they get. They are both complex in their own right. 

For the customer it could be everything in the package that give them an overwhelming satisfaction vs the business needs a value that is good ground to make profit on and keep customers coming back.

---

### How do we define a Bronze / Silver / Gold / Platinum customer?

In [5]:
btl = """ 
SELECT 
    age, 
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 100
ORDER BY 
    total_flights DESC
"""
bronze_tier_lists = pd.read_sql(btl, conn)
bronze_tier_lists

,age,customer_type,flight_distance,total_flights
0,39,Loyal Customer,1427.891045,2937
1,41,Loyal Customer,1412.283409,2851
2,40,Loyal Customer,1450.104596,2763
3,44,Loyal Customer,1441.765221,2743
4,47,Loyal Customer,1407.817675,2693
...,...,...,...,...
61,12,Loyal Customer,946.940086,701
62,8,Loyal Customer,855.043165,695
63,7,Loyal Customer,811.867378,656
64,72,Loyal Customer,1497.981132,212


In [6]:
stl = """ 
SELECT 
    age, 
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 500
ORDER BY 
    total_flights DESC
LIMIT 10
"""

silver_tier_lists = pd.read_sql(stl, conn)
silver_tier_lists

,age,customer_type,flight_distance,total_flights
0,39,Loyal Customer,1427.891045,2937
1,41,Loyal Customer,1412.283409,2851
2,40,Loyal Customer,1450.104596,2763
3,44,Loyal Customer,1441.765221,2743
4,47,Loyal Customer,1407.817675,2693
5,42,Loyal Customer,1464.282520,2683
6,43,Loyal Customer,1471.296863,2678
7,48,Loyal Customer,1416.160211,2659
8,45,Loyal Customer,1399.634441,2648
9,46,Loyal Customer,1454.561999,2621


In [7]:
gtl = """ 
SELECT 
    age, 
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 1000
ORDER BY 
    total_flights DESC
LIMIT 10
"""

gold_tier_list = pd.read_sql(gtl, conn)
gold_tier_list

,age,customer_type,flight_distance,total_flights
0,39,Loyal Customer,1427.891045,2937
1,41,Loyal Customer,1412.283409,2851
2,40,Loyal Customer,1450.104596,2763
3,44,Loyal Customer,1441.765221,2743
4,47,Loyal Customer,1407.817675,2693
5,42,Loyal Customer,1464.282520,2683
6,43,Loyal Customer,1471.296863,2678
7,48,Loyal Customer,1416.160211,2659
8,45,Loyal Customer,1399.634441,2648
9,46,Loyal Customer,1454.561999,2621


In [8]:
ptl = """ 
SELECT 
    age
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 2000
ORDER BY 
    total_flights DESC
LIMIT 10
"""

platinum_tier_list = pd.read_sql(ptl, conn)
platinum_tier_list

,customer_type,flight_distance,total_flights
0,39,1427.891045,2937
1,41,1412.283409,2851
2,40,1450.104596,2763
3,44,1441.765221,2743
4,47,1407.817675,2693
5,42,1464.282520,2683
6,43,1471.296863,2678
7,48,1416.160211,2659
8,45,1399.634441,2648
9,46,1454.561999,2621


### Analysis

The best way to DEFINE tier lists from the dataset we have is to do a count of how much do most age group buy flights and how far do they go on average. I did a filter of more than 100 for the bronze list and we can continue to raise the filter to make it more exclusive for the more frequent flyers.

---

### Do service ratings differ across these simulated tiers?

In [9]:
bsr = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    AVG(on_board_service) AS on_board_service,
    AVG(leg_room_service) AS leg_room_service,
    AVG(food_and_drink) AS food_and_drink,
    AVG(checkin_service) AS checkin_service,
    AVG(inflight_service) AS inflight_service
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 100
ORDER BY 
    total_flights ASC
LIMIT 10
"""

bronze_service_ratings = pd.read_sql(bsr, conn)
bronze_service_ratings

,age,customer_type,flight_distance,total_flights,on_board_service,leg_room_service,food_and_drink,checkin_service,inflight_service
0,71,Loyal Customer,1328.140187,107,3.000000,3.009346,2.906542,2.738318,3.093458
1,72,Loyal Customer,1497.981132,212,2.952830,3.004717,2.933962,2.698113,3.061321
2,7,Loyal Customer,811.867378,656,3.236280,3.181402,2.998476,3.295732,3.757622
3,8,Loyal Customer,855.043165,695,3.303597,3.241727,2.955396,3.315108,3.705036
4,12,Loyal Customer,946.940086,701,3.251070,3.292439,2.911555,3.216833,3.711840
5,13,Loyal Customer,945.518934,713,3.311360,3.231417,3.077139,3.311360,3.702665
6,10,Loyal Customer,903.865734,715,3.247552,3.222378,3.096503,3.250350,3.734266
7,11,Loyal Customer,927.285315,715,3.302098,3.204196,2.983217,3.311888,3.760839
8,9,Loyal Customer,932.696118,747,3.194110,3.191432,2.907631,3.131191,3.673360
9,70,Loyal Customer,879.030790,747,3.115127,2.821954,3.117805,3.270415,3.350736


In [10]:
ssr = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    AVG(on_board_service) AS on_board_service,
    AVG(leg_room_service) AS leg_room_service,
    AVG(food_and_drink) AS food_and_drink,
    AVG(checkin_service) AS checkin_service,
    AVG(inflight_service) AS inflight_service
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 500
ORDER BY 
    total_flights ASC
LIMIT 10
"""

silver_service_ratings = pd.read_sql(ssr, conn)
silver_service_ratings

,age,customer_type,flight_distance,total_flights,on_board_service,leg_room_service,food_and_drink,checkin_service,inflight_service
0,7,Loyal Customer,811.867378,656,3.236280,3.181402,2.998476,3.295732,3.757622
1,8,Loyal Customer,855.043165,695,3.303597,3.241727,2.955396,3.315108,3.705036
2,12,Loyal Customer,946.940086,701,3.251070,3.292439,2.911555,3.216833,3.711840
3,13,Loyal Customer,945.518934,713,3.311360,3.231417,3.077139,3.311360,3.702665
4,10,Loyal Customer,903.865734,715,3.247552,3.222378,3.096503,3.250350,3.734266
5,11,Loyal Customer,927.285315,715,3.302098,3.204196,2.983217,3.311888,3.760839
6,9,Loyal Customer,932.696118,747,3.194110,3.191432,2.907631,3.131191,3.673360
7,70,Loyal Customer,879.030790,747,3.115127,2.821954,3.117805,3.270415,3.350736
8,14,Loyal Customer,919.580132,755,3.247682,3.186755,3.017219,3.219868,3.696689
9,69,Loyal Customer,892.084399,782,3.107417,2.907928,3.094629,3.152174,3.295396


In [11]:
gsr = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    AVG(on_board_service) AS on_board_service,
    AVG(leg_room_service) AS leg_room_service,
    AVG(food_and_drink) AS food_and_drink,
    AVG(checkin_service) AS checkin_service,
    AVG(inflight_service) AS inflight_service
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 1000
ORDER BY 
    total_flights ASC
LIMIT 10
"""
gold_service_ratings = pd.read_sql(gsr, conn)
gold_service_ratings

,age,customer_type,flight_distance,total_flights,on_board_service,leg_room_service,food_and_drink,checkin_service,inflight_service
0,61,Loyal Customer,1052.624192,1083,3.214220,3.041551,3.138504,3.265928,3.405355
1,24,Loyal Customer,1243.219653,1384,3.219653,3.195087,3.276012,3.183526,3.598988
2,22,Loyal Customer,1281.328520,1385,3.168231,3.161733,3.283755,3.119856,3.595668
3,23,Loyal Customer,1270.479697,1453,3.168617,3.165864,3.302822,3.221610,3.625602
4,26,Loyal Customer,1286.521003,1476,3.227642,3.283198,3.369241,3.310976,3.700542
5,28,Loyal Customer,1293.095461,1498,3.324433,3.240988,3.355140,3.293725,3.686248
6,27,Loyal Customer,1345.165449,1505,3.230565,3.203322,3.372093,3.285050,3.672425
7,32,Loyal Customer,1316.983607,1647,3.242259,3.207043,3.409229,3.236794,3.667274
8,29,Loyal Customer,1394.790643,1710,3.236842,3.183041,3.325146,3.276608,3.669006
9,34,Loyal Customer,1322.227877,1729,3.471371,3.469057,3.190862,3.281666,3.681897


In [12]:
psr = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    AVG(on_board_service) AS on_board_service,
    AVG(leg_room_service) AS leg_room_service,
    AVG(food_and_drink) AS food_and_drink,
    AVG(checkin_service) AS checkin_service,
    AVG(inflight_service) AS inflight_service
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 2000
ORDER BY 
    total_flights DESC
LIMIT 10
"""
platinum_service_ratings = pd.read_sql(psr, conn)
platinum_service_ratings

,age,customer_type,flight_distance,total_flights,on_board_service,leg_room_service,food_and_drink,checkin_service,inflight_service
0,39,Loyal Customer,1427.891045,2937,3.554988,3.594144,3.276473,3.372489,3.702417
1,41,Loyal Customer,1412.283409,2851,3.572431,3.585759,3.305507,3.390389,3.719046
2,40,Loyal Customer,1450.104596,2763,3.556279,3.638075,3.301484,3.424177,3.722041
3,44,Loyal Customer,1441.765221,2743,3.561065,3.539920,3.306963,3.395552,3.641269
4,47,Loyal Customer,1407.817675,2693,3.584107,3.529150,3.310434,3.424805,3.644634
5,42,Loyal Customer,1464.282520,2683,3.591502,3.525158,3.262020,3.413343,3.661200
6,43,Loyal Customer,1471.296863,2678,3.538462,3.549291,3.290889,3.422704,3.630695
7,48,Loyal Customer,1416.160211,2659,3.561113,3.541933,3.262881,3.438511,3.651749
8,45,Loyal Customer,1399.634441,2648,3.577795,3.547961,3.283233,3.396526,3.658233
9,46,Loyal Customer,1454.561999,2621,3.572682,3.560855,3.251049,3.425029,3.684090


### Analysis 

The service ratings tend to increase overall as the total flights increase. 

---

### Which tier contains the highest dissatisfaction?

In [13]:
btd = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    COUNT(CASE WHEN satisfaction = 'neutral or disatisfied' THEN 1.0 ELSE 0 END) as total_dissatisfaction
FROM    
    all_customers_loyalty
WHERE 
    customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 100
ORDER BY 
    total_flights ASC
LIMIT 10
"""

bronze_tier_disatisfaction = pd.read_sql(btd, conn)
bronze_tier_disatisfaction

,age,customer_type,flight_distance,total_flights,total_dissatisfaction
0,71,Loyal Customer,1328.140187,107,107
1,72,Loyal Customer,1497.981132,212,212
2,7,Loyal Customer,811.867378,656,656
3,8,Loyal Customer,855.043165,695,695
4,12,Loyal Customer,946.940086,701,701
5,13,Loyal Customer,945.518934,713,713
6,10,Loyal Customer,903.865734,715,715
7,11,Loyal Customer,927.285315,715,715
8,9,Loyal Customer,932.696118,747,747
9,70,Loyal Customer,879.030790,747,747


In [14]:
std = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    COUNT(CASE WHEN satisfaction = 'neutral or disatisfied' THEN 1.0 ELSE 0 END) as total_satisfaction
FROM    
    all_customers_loyalty
WHERE 
      satisfaction = 'neutral or dissatisfied'
      AND customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 500
ORDER BY 
    total_flights ASC
LIMIT 10
"""
silver_tier_dissatisfaction = pd.read_sql(std, conn)
silver_tier_dissatisfaction

,age,customer_type,flight_distance,total_flights,total_satisfaction
0,12,Loyal Customer,898.902230,583,583
1,7,Loyal Customer,808.929174,593,593
2,13,Loyal Customer,885.367720,601,601
3,11,Loyal Customer,900.198686,609,609
4,8,Loyal Customer,828.852273,616,616
5,10,Loyal Customer,883.256778,627,627
6,9,Loyal Customer,915.301391,647,647
7,70,Loyal Customer,831.473118,651,651
8,14,Loyal Customer,870.879573,656,656
9,19,Loyal Customer,885.756469,657,657


In [15]:
gtd = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    COUNT(CASE WHEN satisfaction = 'neutral or disatisfied' THEN 1.0 ELSE 0 END) as total_dissatisfaction
FROM    
    all_customers_loyalty
WHERE 
      satisfaction = 'neutral or dissatisfied'
      AND customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 1000
ORDER BY 
    total_flights ASC
LIMIT 10
"""
gold_tier_dissatisfied = pd.read_sql(gtd, conn)
gold_tier_dissatisfied

,age,customer_type,flight_distance,total_flights,total_dissatisfaction
0,31,Loyal Customer,1141.574850,1002,1002
1,51,Loyal Customer,1037.899202,1002,1002
2,52,Loyal Customer,1018.454455,1010,1010
3,33,Loyal Customer,1056.364985,1011,1011
4,25,Loyal Customer,1103.964532,1015,1015
5,45,Loyal Customer,1016.765686,1020,1020
6,43,Loyal Customer,1101.881605,1022,1022
7,38,Loyal Customer,1041.197073,1025,1025
8,40,Loyal Customer,1074.768031,1026,1026
9,35,Loyal Customer,1035.084548,1029,1029


In [16]:
ptd = """ 
SELECT 
    age,
    customer_type,
    AVG(flight_distance) AS flight_distance,
    COUNT(*) AS total_flights, 
    COUNT(CASE WHEN satisfaction = 'neutral or disatisfied' THEN 1.0 ELSE 0 END) as total_satisfaction
FROM    
    all_customers_loyalty
WHERE 
      satisfaction = 'neutral or dissatisfied'
      AND customer_type = 'Loyal Customer'
GROUP BY 
    age
HAVING 
    total_flights >= 2000
ORDER BY 
    total_flights DESC
LIMIT 10
"""
platinum_tier_dissatisfied = pd.read_sql(ptd, conn)
platinum_tier_dissatisfied

,age,customer_type,flight_distance,total_flights,total_satisfaction


### Analysis

The tier group with the most dissatisfaction appears to be the gold group due to them flying more.

---

### Are high-value customers (long flight_distance) more satisfied?

In [17]:
hvcs = """ 
SELECT 
    age, 
    AVG(flight_distance) AS flight_distance, 
    satisfaction
FROM 
    all_customers_loyalty
GROUP BY
    age
HAVING 
    flight_distance >= 1000
ORDER BY 
    flight_distance DESC
LIMIT 10
"""

high_value_customer_satisfaction = pd.read_sql(hvcs, conn)
high_value_customer_satisfaction

,age,flight_distance,satisfaction
0,77,1450.433962,satisfied
1,46,1439.652300,satisfied
2,43,1404.404292,satisfied
3,49,1384.110112,satisfied
4,72,1358.270161,neutral or dissatisfied
5,44,1353.720878,satisfied
6,45,1329.047147,satisfied
7,28,1186.035497,satisfied
8,35,1184.214226,neutral or dissatisfied
9,78,1181.840909,satisfied


### Analysis

Customers flying longer distances(high value customers) appear to be more satisfied overall than dissatisfied.

---

### Do business travelers produce higher service ratings?

In [18]:
btsr = """ 
SELECT 
    customer_type,
    type_of_travel,
    COUNT(*) AS total_flights, 
    AVG(on_board_service) AS on_board_service,
    AVG(leg_room_service) AS leg_room_service,
    AVG(food_and_drink) AS food_and_drink,
    AVG(checkin_service) AS checkin_service,
    AVG(inflight_service) AS inflight_service
FROM    
    all_customers_loyalty
GROUP BY 
    type_of_travel
ORDER BY 
    total_flights DESC
LIMIT 10
"""

business_travel_service_ratings = pd.read_sql(btsr, conn)
business_travel_service_ratings

,customer_type,type_of_travel,total_flights,on_board_service,leg_room_service,food_and_drink,checkin_service,inflight_service
0,Loyal Customer,Business travel,89445,3.434613,3.473956,3.265839,3.292235,3.660808
1,Loyal Customer,Personal Travel,40042,3.268368,3.076595,3.068079,3.337521,3.601194


### Analysis

Business travelers produce way higher service ratings except in the checkin service. 

We need to find a way to bring personal travelers the same satisfaction as business travelers but we also do not want to give up the benefits of traveling business.

---